# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 12:51PM,246725,16,8546450,Sartorius Bioprocess Solutions,Released
1,Sep 19 2022 12:51PM,246724,16,8546445,Sartorius Bioprocess Solutions,Released
2,Sep 19 2022 12:50PM,246723,16,8546365,Sartorius Bioprocess Solutions,Released
3,Sep 19 2022 12:41PM,246712,15,8545130,"Brookfield Pharmaceuticals, LLC",Released
4,Sep 19 2022 12:41PM,246712,15,8545144,"Brookfield Pharmaceuticals, LLC",Released
5,Sep 19 2022 12:41PM,246712,15,8545145,"Brookfield Pharmaceuticals, LLC",Released
6,Sep 19 2022 12:41PM,246712,15,8545146,"Brookfield Pharmaceuticals, LLC",Released
7,Sep 19 2022 12:41PM,246712,15,8545190,"Brookfield Pharmaceuticals, LLC",Released
8,Sep 19 2022 12:41PM,246712,15,8545191,"Brookfield Pharmaceuticals, LLC",Released
9,Sep 19 2022 12:41PM,246712,15,8545698,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,246721,Executing,4
19,246721,Released,19
20,246723,Released,1
21,246724,Released,1
22,246725,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246720,NaN,1.0,2.0
246721,NaN,4.0,19.0
246723,NaN,NaN,1.0
246724,NaN,NaN,1.0
246725,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0.0,1.0,0.0
246693,0.0,1.0,0.0
246697,5.0,11.0,3.0
246700,0.0,7.0,11.0
246701,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0,1,0
246693,0,1,0
246697,5,11,3
246700,0,7,11
246701,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,0,1,0
1,246693,0,1,0
2,246697,5,11,3
3,246700,0,7,11
4,246701,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,,1,
1,246693,,1,
2,246697,5,11,3
3,246700,,7,11
4,246701,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC"
67,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd"
90,Sep 19 2022 12:21PM,246720,19,"GUSA Granules USA, Inc."
93,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation
100,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc."
102,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc."
105,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions,,,1
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions,,,1
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions,,,1
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC",,,64
4,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd",,4,19
5,Sep 19 2022 12:21PM,246720,19,"GUSA Granules USA, Inc.",,1,2
6,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,,,7
7,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",,,2
8,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",,1,2
9,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,,8,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions,1,,
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions,1,,
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions,1,,
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC",64,,
4,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd",19,4,
5,Sep 19 2022 12:21PM,246720,19,"GUSA Granules USA, Inc.",2,1,
6,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,7,,
7,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",2,,
8,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",2,1,
9,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,1,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions,1,,
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions,1,,
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions,1,,
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC",64,,
4,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd",19,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC",64.0,NaN,NaN
4,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd",19.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 12:51PM,246725,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Sep 19 2022 12:51PM,246724,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Sep 19 2022 12:50PM,246723,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Sep 19 2022 12:41PM,246712,15,"Brookfield Pharmaceuticals, LLC",64.0,0.0,0.0
4,Sep 19 2022 12:33PM,246721,19,"AdvaGen Pharma, Ltd",19.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1480251,42.0,15.0,0.0
15,493422,66.0,1.0,0.0
16,1233542,3.0,2.0,0.0
19,493441,21.0,5.0,0.0
20,246697,3.0,11.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1480251,42.0,15.0,0.0
1,15,493422,66.0,1.0,0.0
2,16,1233542,3.0,2.0,0.0
3,19,493441,21.0,5.0,0.0
4,20,246697,3.0,11.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,15.0,0.0
1,15,66.0,1.0,0.0
2,16,3.0,2.0,0.0
3,19,21.0,5.0,0.0
4,20,3.0,11.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,15,Released,66.0
2,16,Released,3.0
3,19,Released,21.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,0.0,0.0,0.0,0.0,5.0
Executing,15.0,1.0,2.0,5.0,11.0
Released,42.0,66.0,3.0,21.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,5.0
1,Executing,15.0,1.0,2.0,5.0,11.0
2,Released,42.0,66.0,3.0,21.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,5.0
1,Executing,15.0,1.0,2.0,5.0,11.0
2,Released,42.0,66.0,3.0,21.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()